In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

## Task 1] Build the optimisation model

In [2]:
# Load the data from the data folder
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv', delimiter=';')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv', delimiter=';')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv', delimiter=';')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv', delimiter=';')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv', delimiter=';')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv', delimiter=';')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv', delimiter=';')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv', delimiter=';')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv', delimiter=';')


In [3]:
Nodes = ['Node 1', 'Node 2', 'Node 3', 'Node 4', 'Node 5', 'Node 6']
Generator = ['G1', 'G2', 'G3']
Generator_node = {'Node 1': 'G1', 'Node 2': 'G2', 'Node 6': 'G3'}
Load = ['L1', 'L2', 'L3']
Load_node = {'Node 4': 'L1', 'Node 5': 'L2', 'Node 6': 'L3'}
Wind = ['W1', 'W2']
Wind_node = {'Node 4': 'W1', 'Node 5': 'W2'}
Transmission = ['Line 1', 'Line 2', 'Line 3', 'Line 4', 'Line 5', 'Line 6','Line 7']
Transmission_node = {'Line 1': ['Node 1', 'Node 2'], 'Line 2': ['Node 2', 'Node 3'], 'Line 3': ['Node 3', 'Node 6'], 'Line 4': ['Node 5', 'Node 6'], 'Line 5': ['Node 4', 'Node 5'], 'Line 6': ['Node 2', 'Node 4'],'Line 6': ['Node 1', 'Node 4']}    

In [4]:
# Create matrix with the nodes as columns and the generators, loads and winds as rows, with 1 if connected to the node
Gen_n = np.zeros((len(Generator), len(Nodes)))
Load_n = np.zeros((len(Load), len(Nodes)))
Wind_n = np.zeros((len(Wind), len(Nodes)))
Transmission_n = np.zeros((len(Transmission), len(Nodes)))

# Populate the matrix
for i, g in enumerate(Generator):  # Iterate over generators
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Generator_node.get(node) == g:  # Check if generator is connected to the node
            Gen_n[i, j] = 1

for i, l in enumerate(Load):  # Iterate over loads
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Load_node.get(node) == l:  # Check if load is connected to the node
            Load_n[i, j] = 1

for i, w in enumerate(Wind):  # Iterate over winds
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Wind_node.get(node) == w:  # Check if wind is connected to the node
            Wind_n[i, j] = 1

for i, t in enumerate(Transmission):  # Iterate over transmission lines
    connected_nodes = Transmission_node.get(t, [])  # Get nodes connected by the transmission line
    for node in connected_nodes:  # For each node connected by the transmission line
        if node in Nodes:  # Ensure the node is valid (exists in Nodes list)
            j = Nodes.index(node)  # Get the column index for the node in Transmission_n
            Transmission_n[i, j] = 1  # Set the corresponding element to 1
            


In [5]:
# Define the input data class
class InputData:
    
    def __init__(
        self,
        wind_forecast: pd.DataFrame, 
        bus: pd.DataFrame,
        load: pd.DataFrame,
        max_prod: pd.DataFrame,
        min_prod: pd.DataFrame,
        min_down_time: pd.DataFrame,
        min_up_time: pd.DataFrame,
        prod_cost: pd.DataFrame,
        ramp_rate: pd.DataFrame,
        start_up_cost: pd.DataFrame,
        transmission_cap: pd.DataFrame
    ):
        self.time = range(len(wind_forecast))  #maybe define it with lenght of wind_production
        self.wind_forecast = wind_forecast
        self.bus = bus
        self.load = load
        self.max_prod = max_prod
        self.min_prod = min_prod
        self.min_down_time = min_down_time
        self.min_up_time = min_up_time
        self.prod_cost = prod_cost
        self.ramp_rate = ramp_rate
        self.start_up_cost = start_up_cost
        self.transmission_cap = transmission_cap
        self.M = 1000000  # Penalty for having flexible demand
        self.Gen_n = Gen_n  # Matrix mapping generators to nodes
        self.Load_n = Load_n # Matrix mapping loads to nodes
        self.Wind_n = Wind_n # Matrix mapping wind to nodes
        
        


In [6]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

# GET THE INPUT FROM THE OTHERS

In [7]:
# import the csv files X_val, y_val and y_pred
X_val = pd.read_csv('../Data/X_val.csv', delimiter=',')
y_val = pd.read_csv('../Data/y_val.csv', delimiter=',')
y_pred = pd.read_csv('../Data/y_pred.csv', delimiter=',')

In [8]:
# Add a first column hour to y_pred and y_val
y_pred['hour'] = range(0, len(y_pred))
y_val['hour'] = range(0, len(y_val))
y_pred = y_pred[['hour', 'G1', 'G2', 'G3']]
y_val = y_val[['hour', 'G1', 'G2', 'G3']]

In [9]:
# Define the optimization model class

class EconomicDispatch_Test():
        
        def __init__(self, input_data: InputData, y_pred: pd.DataFrame):
            self.data = input_data 
            self.y_pred = y_pred.T
            self.variables = Expando()
            self.constraints = Expando() 
            self.results = Expando() 
            self._build_model() 
            
        def _build_variables(self):
            # one variable for each generator for each time of the day
            self.variables.prod_gen = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.max_prod.iloc[i-1, 0], 
                                           name='generation_G{}_{}'.format(i, t)) 
                                           for i in range(1, len(self.data.max_prod)+1) 
                                           for t in self.data.time}
            
            # one variable for each wind generator for each time of the day
            self.variables.prod_wind = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.wind_forecast.iloc[t, i], 
                                            name='wind_generation_W{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.wind_forecast.iloc[0, :])) 
                                            for t in self.data.time}
            
            # one variable for each start-up cost for each generator
            self.variables.start_up_cost = {
                 (i, t): self.model.addVar(lb=0, 
                                            name='start_up_cost_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # add two slack variables to always make the model feasible, allowing the demand to be flexible
            self.variables.epsilon = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='epsilon_Bus{}_{}'.format(n, t)) 
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1) 
                                           for t in self.data.time}
            self.variables.delta = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='delta_Bus{}_{}'.format(n, t))
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                           for t in self.data.time}
            
            # add two slack variables to always make the model feasible, relaxing the min_up_time and min_down_time constraints
            self.variables.alpha = {
                    (i, t, to): self.model.addVar(lb=0,
                                            name='alpha_G{}_{}_{}'.format(i, t, to))
                                            for i in range(1, len(self.data.max_prod)+1)
                                            for t in self.data.time
                                            for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            self.variables.beta = {
                    (i, t, to): self.model.addVar(lb=0,
                                            name='beta_G{}_{}_{}'.format(i, t, to))
                                            for i in range(1, len(self.data.max_prod)+1)
                                            for t in self.data.time
                                            for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}

            
            
        def _build_constraints(self):
            # Minimum capacity of the generator
            self.constraints.min_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] >= self.data.min_prod.iloc[i-1, 0] * self.y_pred.iloc[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}
            # Maximum capacity of the generator
            self.constraints.max_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] <= self.data.max_prod.iloc[i-1, 0] * self.y_pred.iloc[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}

            # Power balance constraint
            self.constraints.power_balance = {
                t: self.model.addConstr(
                    gp.quicksum(self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod) + 1)) +
                    gp.quicksum(self.variables.prod_wind[i, t] for i in range(1, len(self.data.wind_forecast.iloc[0, :]))) == 
                    gp.quicksum(self.data.load.iloc[t, i] * Load_n[i-1, n-1] for i in range(1, len(self.data.load.iloc[0, :]))for n in range(1, len(self.data.bus.iloc[0, :]) + 1))
                    + gp.quicksum(self.variables.epsilon[n, t] - self.variables.delta[n, t] for n in range(1, len(self.data.bus.iloc[0, :]) + 1))
                ) for t in self.data.time}
        
            # Transmission capacity constraint up
            self.constraints.transmission_capacity_up = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(
                            self.data.bus.iloc[l-1, n-1] * Transmission_n[l-1, n-1] * (
                                self.variables.prod_gen[g, t] * Gen_n[g-1, n-1] +
                                self.variables.prod_wind[w, t] * Wind_n[w-1, n-1] -
                                self.data.load.iloc[t, i] * Load_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]
                            )
                            for n in range(1, len(self.data.bus.iloc[0, :]) + 1)
                            for i in range(1, len(self.data.load.iloc[0, :]))
                            for g in range(1, len(self.data.max_prod) + 1)
                            for w in range(1, len(self.data.wind_forecast.iloc[0, :]))
                        ) <= self.data.transmission_cap.iloc[l-1, 0],
                        name="transmission_capacity_up_L{}_T{}".format(l, t)
                    ) for l in range(1, len(self.data.transmission_cap) + 1)
                    for t in self.data.time
                }

            #Transmission capacity constraint down
            self.constraints.transmission_capacity_down = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(
                            self.data.bus.iloc[l-1, n-1] * Transmission_n[l-1, n-1] * (
                                self.variables.prod_gen[g, t] * Gen_n[g-1, n-1] +
                                self.variables.prod_wind[w, t] * Wind_n[w-1, n-1] -
                                self.data.load.iloc[t, i] * Load_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]
                            )
                            for n in range(1, len(self.data.bus.iloc[0, :]) + 1)
                            for i in range(1, len(self.data.load.iloc[0, :]))
                            for g in range(1, len(self.data.max_prod) + 1)
                            for w in range(1, len(self.data.wind_forecast.iloc[0, :]))
                        ) >= -self.data.transmission_cap.iloc[l-1, 0],
                        name="transmission_capacity_down_L{}_T{}".format(l, t)
                    ) for l in range(1, len(self.data.transmission_cap) + 1)
                    for t in self.data.time
                }

            #Start-up costs constraint
            self.constraints.start_up_cost = {
                (i, t): self.model.addConstr(
                    self.variables.start_up_cost[i, t] >= self.data.start_up_cost.iloc[i-1, 0] * (self.y_pred.iloc[i, t] - self.y_pred.iloc[i, t-1])
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.start_up_cost_0 = {
                i: self.model.addConstr(
                    self.variables.start_up_cost[i, 0] >= self.data.start_up_cost.iloc[i-1, 0] * self.y_pred.iloc[i, 0]
                ) for i in range(1, len(self.data.max_prod)+1)}
            
            # Ramping constraint
            self.constraints.ramping_up = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] - self.variables.prod_gen[i, t-1] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.ramping_down = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t-1] - self.variables.prod_gen[i, t] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            
            # Minimum up time constraint
            self.constraints.min_up_time = {
                (i, t, to): self.model.addConstr(
                    -self.y_pred.iloc[i, t - 1] + self.y_pred.iloc[i, t] - self.y_pred.iloc[i, to] - self.variables.alpha[i, t, to] <= 0
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            # Minimum down time constraint
            self.constraints.min_down_time = {
                (i, t, to): self.model.addConstr(
                    self.y_pred.iloc[i, t - 1] - self.y_pred.iloc[i, t] + self.y_pred.iloc[i, to] - self.variables.beta[i, t, to] <= 1
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            


        def _build_objective(self):
            # Objective function
            self.model.setObjective(
                gp.quicksum(self.data.prod_cost.iloc[i-1, 0]*self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                gp.quicksum(self.variables.start_up_cost[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                self.data.M * (gp.quicksum(self.variables.epsilon[n, t] + self.variables.delta[n, t] for n in range(1, len(self.data.bus.iloc[0,:])+1) for t in self.data.time)) +
                self.data.M * (gp.quicksum(self.variables.alpha[i, t, to] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0)) +
                self.data.M * (gp.quicksum(self.variables.beta[i, t, to] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0))
            )

        def _build_model(self):
            self.model = gp.Model('EconomicDispatch')
            self._build_variables()
            self._build_constraints()
            self._build_objective()
            self.model.update()

        def optimize(self):
            self.model.optimize()
            self._extract_results()

        def _extract_results(self):
            self.results.production = pd.DataFrame({
                
                'hour': [self.data.wind_forecast.iloc[t, 0] for t in self.data.time],
                #'time': [t for t in self.data.time],
                #'status G1': [self.y_pred.iloc[1, t] for t in self.data.time],
                #'status G2': [self.y_pred.iloc[2, t] for t in self.data.time],
                #'status G3': [self.y_pred.iloc[3, t] for t in self.data.time],
                #'start_up_cost 1': [self.variables.start_up_cost[1, t].x for t in self.data.time],
                #'start_up_cost 2': [self.variables.start_up_cost[2, t].x for t in self.data.time],
                #'start_up_cost 3': [self.variables.start_up_cost[3, t].x for t in self.data.time],
                #'generation 1': [self.variables.prod_gen[1, t].x for t in self.data.time],
                #'generation 2': [self.variables.prod_gen[2, t].x for t in self.data.time],
                #'generation 3': [self.variables.prod_gen[3, t].x for t in self.data.time],
                #'wind generation 1': [self.variables.prod_wind[1, t].x for t in self.data.time],
                #'wind generation 2': [self.variables.prod_wind[2, t].x for t in self.data.time],
                #'load 1': [self.data.load.iloc[t, 1] for t in self.data.time],
                #'load 2': [self.data.load.iloc[t, 2] for t in self.data.time],
                #'load 3': [self.data.load.iloc[t, 3] for t in self.data.time],
                #'epsilon 1': [self.variables.epsilon[1, t].x for t in self.data.time],
                #'delta 1': [self.variables.delta[1, t].x for t in self.data.time],
                #'epsilon 2': [self.variables.epsilon[2, t].x for t in self.data.time],
                #'delta 2': [self.variables.delta[2, t].x for t in self.data.time],
                #'epsilon 3': [self.variables.epsilon[3, t].x for t in self.data.time],
                #'delta 3': [self.variables.delta[3, t].x for t in self.data.time],
                #'epsilon 4': [self.variables.epsilon[4, t].x for t in self.data.time],
                #'delta 4': [self.variables.delta[4, t].x for t in self.data.time],
                #'epsilon 5': [self.variables.epsilon[5, t].x for t in self.data.time],
                #'delta 5': [self.variables.delta[5, t].x for t in self.data.time],
                #'epsilon 6': [self.variables.epsilon[6, t].x for t in self.data.time],
                #'delta 6': [self.variables.delta[6, t].x for t in self.data.time]
            })

            # Add columns for each transmission line's binding status at each time
            for l in range(1, len(self.data.transmission_cap) + 1):
                up_binding = []
                down_binding = []
                
                for t in self.data.time:
                    up_constraint = self.constraints.transmission_capacity_up[l, t]
                    down_constraint = self.constraints.transmission_capacity_down[l, t]

                    # Append binding status (True if binding, based on slack value)
                    up_binding.append(abs(up_constraint.slack) < 1e-6)
                    down_binding.append(abs(down_constraint.slack) < 1e-6)

                # Add the binding status as new columns in the main production DataFrame
                self.results.production[f'transmission_up_binding_L{l}'] = up_binding
                self.results.production[f'transmission_down_binding_L{l}'] = down_binding

            # Add the alpha and beta variables to the results
            self.results.alpha1 = [gp.quicksum(self.variables.alpha[1, t, to].x  for to in range(t, min(t + self.data.min_up_time.iloc[0, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            self.results.alpha2 = [gp.quicksum(self.variables.alpha[2, t, to].x  for to in range(t, min(t + self.data.min_up_time.iloc[1, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            self.results.alpha3 = [gp.quicksum(self.variables.alpha[3, t, to].x  for to in range(t, min(t + self.data.min_up_time.iloc[2, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            self.results.beta1 = [gp.quicksum(self.variables.beta[1, t, to].x  for to in range(t, min(t + self.data.min_down_time.iloc[0, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            self.results.beta2 = [gp.quicksum(self.variables.beta[2, t, to].x  for to in range(t, min(t + self.data.min_down_time.iloc[1, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            self.results.beta3 = [gp.quicksum(self.variables.beta[3, t, to].x  for to in range(t, min(t + self.data.min_down_time.iloc[2, 0], len(self.data.time))) if t > 0) for t in self.data.time]
            

                                  
            self.results.objective = self.model.objVal
            
        def _print_model(self):
            self.model.write('EconomicDispatch.lp')        
            
                 

In [10]:
# Run the model test
wind_forecast_test = X_val.drop(columns=['L1', 'L2', 'L3']).head(24)
load_test = X_val.drop(columns=['W1', 'W2']).head(24)
input_data = InputData(wind_forecast_test, bus, load_test, max_prod, min_prod, min_down_time, min_up_time, prod_cost, ramp_rate, start_up_cost, transmission_cap)
model_test = EconomicDispatch_Test(input_data, y_val.head(24))
#model_test = EconomicDispatch_Test(input_data, y_pred.head(24))
model_test.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-24


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1043 rows, 809 columns and 2573 nonzeros
Model fingerprint: 0x7e1b2557
Coefficient statistics:
  Matrix range     [4e-01, 1e+01]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e-01, 2e+02]
  RHS range        [1e+00, 9e+02]
Presolve removed 830 rows and 432 columns
Presolve time: 0.01s
Presolved: 213 rows, 559 columns, 1259 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0652360e+06   3.964582e+02   0.000000e+00      0s
      61    1.0094959e+08   0.000000e+00   0.000000e+00      0s

Solved in 61 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.009495877e+08


In [11]:
# Create a dataframe with the sum of the alpha and beta variables
unfeasible_hours = []
for i in range (1, 4):
    alpha = [
        sum(
            model_test.variables.alpha[i, t, to].x
            for to in range(t, min(t + model_test.data.min_up_time.iloc[i-1, 0], len(model_test.data.time)))
        if t > 0
        )
        for t in model_test.data.time
    ]
    unfeasible_hours.append(sum(alpha))
    beta = [
        sum(
            model_test.variables.beta[i, t, to].x
            for to in range(t, min(t + model_test.data.min_down_time.iloc[i-1, 0], len(model_test.data.time)))
        if t > 0
        )
        for t in model_test.data.time
    ]
    unfeasible_hours.append(sum(alpha))

# Print the number of unfeasible hours for each generator
print('Total number of hours:', len(model_test.data.time))
print('Unfeasible hours for generator 1:', unfeasible_hours[0]+ unfeasible_hours[1], ', so', round((unfeasible_hours[0]+ unfeasible_hours[1])/len(model_test.data.time)*100, 2), '% of the total hours')
print('Unfeasible hours for generator 2:', unfeasible_hours[2]+ unfeasible_hours[3], ', so', round((unfeasible_hours[2]+ unfeasible_hours[3])/len(model_test.data.time)*100, 2), '% of the total hours')
print('Unfeasible hours for generator 3:', unfeasible_hours[4]+ unfeasible_hours[5], ', so', round((unfeasible_hours[4]+ unfeasible_hours[5])/len(model_test.data.time)*100, 2), '% of the total hours')


Total number of hours: 24
Unfeasible hours for generator 1: 0.0 , so 0.0 % of the total hours
Unfeasible hours for generator 2: 12.0 , so 50.0 % of the total hours
Unfeasible hours for generator 3: 0.0 , so 0.0 % of the total hours
